## CNN

In [ ]:
from CNN.lenet import LeNet5
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms
import quartz
from quartz import layers
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
model = LeNet5(10)
model.load_state_dict(torch.load("CNN/quartz-lenet.pth", map_location=torch.device('cpu')))
model.eval()

class Lenet1(nn.Module):
            def __init__(self):
                super(Lenet1, self).__init__()
                self.features = nn.Sequential(
                    model.conv1,
                    nn.ReLU(),
                    model.pool1,
                    model.drop1,
#                     model.conv2,
#                     nn.ReLU(),
#                     model.pool2,
#                     model.drop2,
                )
            def forward(self, x):
                x = self.features(x)
                return x

layer = Lenet1()
layer.eval()

In [ ]:
def get_weights_biases(model):
    parameters = list(model.parameters())
    weights = [weight.detach().numpy() for weight in parameters[::2]]
    biases = [bias.detach().numpy() for bias in parameters[1::2]]
    return weights, biases

weights, biases = get_weights_biases(model)

In [ ]:
([weight.max() for weight in weights])
([weight.min() for weight in weights])

## build model

In [ ]:
t_max = 2**8
input_dims = (1,32,32)
pool_kernel_size = [2,2]
batch_size=1
steps_per_sample=1600

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights[0], biases=biases[0]),
    layers.MaxPool2D(kernel_size=pool_kernel_size),
#     layers.Conv2D(weights=weights[1], biases=biases[1]),
#     layers.MaxPool2D(kernel_size=pool_kernel_size),
#     layers.Conv2D(weights=weights[2], biases=biases[2]),
#     layers.Dense(weights=weights[3], biases=biases[3]),
#     layers.Dense(weights=weights[4], biases=biases[4]),
    layers.MonitorLayer(),
])

## load data

In [ ]:
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(),])
test_set = datasets.MNIST('./CNN/data', train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [ ]:
samples, target = next(iter(test_loader))
inputs = quartz.utils.decode_values_into_spike_input(samples.detach().numpy(), t_max, steps_per_sample=steps_per_sample)
target

In [ ]:
loihi_output = loihi_model(samples.detach().numpy(), t_max)

In [ ]:
layer_output = layer(samples)

## convmax model

In [ ]:
loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights[0], biases=biases[0], pool_kernel_size=pool_kernel_size),
#     layers.ConvPool2D(weights=weights[1], biases=biases[1], pool_kernel_size=pool_kernel_size),
#     layers.Conv2D(weights=weights[2], biases=biases[2]),
#     layers.Dense(weights=weights[3], biases=biases[3]),
#     layers.Dense(weights=weights[4], biases=biases[4]),
    layers.MonitorLayer(),
])

In [ ]:
loihi_output1 = loihi_model1(samples.detach().numpy(), t_max)

In [ ]:
assert len(loihi_output) == len(layer_output.flatten())

In [ ]:
combinations = list(zip(loihi_output, layer_output.detach().cpu().numpy().flatten()))

In [ ]:
combinations1 = list(zip(loihi_output1, layer_output.detach().cpu().numpy().flatten()))

In [ ]:
combinations2 = list(zip(loihi_output, loihi_output1))

In [ ]:
len(combinations2)

In [ ]:
sum(loihi_output == loihi_output1)

In [ ]:
loihi_output[loihi_output != loihi_output1]

In [ ]:
combinations1